In [1]:
import pandas as pd
import numpy as np
from neo4j import GraphDatabase
from neo4j.exceptions import ServiceUnavailable
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from transformers import pipeline

### CONFIGURATION ###
NEO4J_URI = "bolt://localhost:7687"
NEO4J_USER = "neo4j"
NEO4J_PASSWORD = "argentic"  




1. Connect to Neo4j and Retrieve Nodes

In [2]:

try:
    driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))
    # Test connection
    with driver.session() as session:
        session.run("RETURN 1")
    print("Connected to Neo4j successfully.")
except ServiceUnavailable as e:
    print("Neo4j connection error:", e)
    exit(1)

def get_nodes(label):
    with driver.session() as session:
        result = session.run(f"MATCH (n:{label}) RETURN n")
        nodes = [record["n"] for record in result]
    return nodes

# Retrieve nodes from the graph.
cities = get_nodes("City")
flights = get_nodes("Flight")
hotels = get_nodes("Hotel")
restaurants = get_nodes("Restaurant")
preferences = get_nodes("Preference")
users = get_nodes("User")

Connected to Neo4j successfully.


# ----------------------------
# 2. Create Clean Text Representations for Each Node
# ----------------------------

In [3]:


def build_representation(props, fields):
    parts = []
    for field, label in fields.items():
        value = props.get(field)
        if value is not None and str(value).strip() != "":
            parts.append(f"{label}: {value}")
    return "; ".join(parts)

def represent_city(node):
    fields = {
        "City": "City",
        "Country": "Country",
        "weather": "Weather",
        "avg_flight_cost": "Avg Flight Cost",
        "avg_hotel_cost": "Avg Hotel Cost"
    }
    return build_representation(node._properties, fields)

def represent_flight(node):
    fields = {
        "Flight": "Flight",
        "Departure": "Departure",
        "Arrival": "Arrival",
        "avg_cost": "Avg Cost"
    }
    return build_representation(node._properties, fields)

def represent_hotel(node):
    fields = {
        "Hotel": "Hotel",
        "City": "City",
        "Country": "Country",
        "avg_cost": "Avg Cost"
    }
    return build_representation(node._properties, fields)

def represent_restaurant(node):
    fields = {
        "Restaurant": "Restaurant",
        "City": "City",
        "Country": "Country",
        "rating": "Rating"
    }
    return build_representation(node._properties, fields)

def represent_preference(node):
    fields = {
        "user_id": "User ID",
        "preferences": "Preferences"
    }
    return build_representation(node._properties, fields)

def represent_user(node):
    fields = {
        "user_id": "User ID",
        "name": "Name"
    }
    return build_representation(node._properties, fields)

def get_clean_representations(nodes, represent_func):
    reps = []
    for node in nodes:
        rep = represent_func(node)
        if rep and "Unknown" not in rep:
            reps.append(rep)
    return reps

representations = []
representations += get_clean_representations(cities, represent_city)
representations += get_clean_representations(flights, represent_flight)
representations += get_clean_representations(hotels, represent_hotel)
representations += get_clean_representations(restaurants, represent_restaurant)
representations += get_clean_representations(preferences, represent_preference)
representations += get_clean_representations(users, represent_user)

if not representations:
    print("Warning: No clean representations found; using all available representations.")
    for node in cities:
        representations.append(represent_city(node))
    for node in flights:
        representations.append(represent_flight(node))
    for node in hotels:
        representations.append(represent_hotel(node))
    for node in restaurants:
        representations.append(represent_restaurant(node))
    for node in preferences:
        representations.append(represent_preference(node))
    for node in users:
        representations.append(represent_user(node))

representations = list(set(representations))
print(f"Total representations for retrieval: {len(representations)}")



Total representations for retrieval: 1000


# ----------------------------
# 3. Compute Embeddings for All Representations
# ----------------------------

In [4]:
print("Computing embeddings...")
embedder = SentenceTransformer("all-MiniLM-L6-v2")
doc_embeddings = embedder.encode(representations, convert_to_tensor=True)

Computing embeddings...


# ----------------------------
# 4. Define a Retrieval Function
# ----------------------------

In [5]:
def retrieve_documents(query, top_k=3):
    query_embedding = embedder.encode([query], convert_to_tensor=True)
    cos_scores = cosine_similarity(query_embedding.cpu().numpy(), doc_embeddings.cpu().numpy())[0]
    top_indices = np.argsort(cos_scores)[::-1][:top_k]
    retrieved_docs = [representations[i] for i in top_indices]
    return retrieved_docs

# ----------------------------
# 5. LLM for Final Answer Generation (Streamlined Prompt)
# ----------------------------

In [6]:
generator = pipeline("text-generation", model="gpt2", max_length=150)

def generate_final_answer(query):
    retrieved_docs = retrieve_documents(query, top_k=3)
    # Build a clean prompt without repeating extraneous information.
    prompt = "Travel Data:\n" + "\n".join(retrieved_docs) + "\n\n"
    prompt += f"Question: {query}\nAnswer:"
    result = generator(prompt, max_length=150, num_return_sequences=1)
    generated_text = result[0]["generated_text"]
    # Remove the prompt from the output.
    answer = generated_text.replace(prompt, "").strip()
    # Optionally, split and take the first line if it contains extra information.
    answer = answer.split("\n")[0].strip()
    return answer


Device set to use cpu


# ----------------------------
# 6. Process Sample Questions and Save to CSV
# ----------------------------

In [7]:
sample_questions = [
    "What country is Miami in and what are the average travel costs there?",
    "List cities with sunny weather and affordable flight and hotel costs.",
    "What is the average flight cost to Paris?",
    "Which city offers last-minute travel deals?",
    "Find restaurants in New York with high ratings.",
    "What is the average hotel cost in Tokyo?",
    "Which flight provides the best balance of cost and travel time?",
    "List cities with rainy weather.",
    "What are common user travel preferences?",
    "Which city has a high cultural score and low hotel cost?",
    "Find flights departing from New York with low average cost.",
    "Which hotel in Paris has the best reviews?",
    "List restaurants in San Francisco with a rating above 4.5.",
    "Which city in Europe is known for its sunny weather?",
    "Find travel options for budget-conscious travelers.",
    "Which user has expressed interest in cultural attractions?",
    "List cities with a high average flight cost.",
    "Which flight has the shortest travel time?",
    "Find hotels with an average cost below $100.",
    "Which restaurant in Rome is best known for its authentic cuisine?",
    "List all cities offering last-minute deals.",
    "What is the average hotel cost in New York?",
    "Which city has the highest cultural score?",
    "Find flights with the best customer reviews.",
    "Which hotel offers the best value for money in London?",
    "List restaurants with diverse cuisine options in Los Angeles.",
    "Which flight is most popular among users?",
    "Find cities with both low flight and hotel costs.",
    "Which restaurant has the highest rating overall?",
    "List travel options for luxury seekers.",
    "Which city is known for its diverse cultural heritage?",
    "Find budget hotels in Miami.",
    "Which flight has the best on-time performance?",
    "List restaurants that are popular among locals.",
    "Which city offers the best combination of weather and cultural attractions?",
    "Find flights that operate internationally with low cost.",
    "Which hotel in Tokyo is recommended for business travelers?",
    "List cities with high tourist satisfaction ratings.",
    "Which restaurant offers the best outdoor dining experience?",
    "Find travel options that emphasize eco-friendly practices.",
    "Which flight offers flexible cancellation policies?",
    "List hotels that have received awards for service excellence.",
    "Which city has the best public transportation system for tourists?",
    "Find restaurants that offer vegan options.",
    "Which hotel in Paris offers the most affordable luxury experience?",
    "List cities with a growing number of international flights.",
    "Which flight offers the best connectivity between major hubs?",
    "Find travel packages that include both flight and hotel discounts.",
    "Which city has the best nightlife for travelers?",
    "Find budget-friendly restaurants in Chicago."
]

output_rows = []
for q in sample_questions:
    answer = generate_final_answer(q)
    output_rows.append({"Question": q, "Answer": answer})
    print("Q:", q)
    print("A:", answer)
    print("-----")

results_df = pd.DataFrame(output_rows)
results_df.to_csv("generated_travel_queries.csv", index=False)
print("CSV file 'generated_travel_queries.csv' created with question-answer pairs.")

# Close the Neo4j driver
driver.close()


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Q: What country is Miami in and what are the average travel costs there?
A: In Miami, $1.2 billion a year, and in other major cities like Boston, Philadelphia, New York, Chicago (including those
-----


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Q: List cities with sunny weather and affordable flight and hotel costs.
A: Answer #2: Sunny days are not generally associated with better flight times for passengers than daytime on average.
-----


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


...

Q: Which flight offers the best connectivity between major hubs?
A: Seattle (2nd most congested, 2nd most landlocked) versus Dallas and Portland (3rd least congested, 3rd least landlocked).
-----


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Q: Find travel packages that include both flight and hotel discounts.
A: We have several, but we aren't looking at flights you can get or hotel discounts.
-----


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Q: Which city has the best nightlife for travelers?
A: Minneapolis Minneapolis Minneapolis
-----
Q: Find budget-friendly restaurants in Chicago.
A: Chicago is known for a lot of high quality food: it has a lot of good restaurants (like the $27.99 Chicken Fried Chicken and $36.
-----
CSV file 'generated_travel_queries.csv' created with question-answer pairs.
